In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import torch
import pytorch_lightning as pl
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T
from ptls.preprocessing import PysparkDataPreprocessor

In [3]:
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T

In [ ]:
data_path = 'data/'

spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("PysparkDataPreprocessor")
spark_conf.set("spark.driver.maxResultSize", "16g")
spark_conf.set("spark.executor.memory", "32g")
spark_conf.set("spark.executor.memoryOverhead", "16g")
spark_conf.set("spark.driver.memory", "32g")
spark_conf.set("spark.driver.memoryOverhead", "16g")
spark_conf.set("spark.cores.max", "24")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

# Trx

In [6]:
trx_train = spark.read.parquet('trx_train.parquet')
trx_test = spark.read.parquet('trx_test.parquet')

In [ ]:
preprocessor_trx = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="event_time",
    event_time_transformation="dt_to_timestamp",
    cols_category=[
        "event_type",
        "event_subtype",
        "currency",
        "src_type11",
        "src_type12",
        "dst_type11",
        "dst_type12",
        "src_type21",
        "src_type22",
        "src_type31",
        "src_type32"
    ],
    cols_identity="amount",
)

processed_train = preprocessor_trx.fit_transform(trx_train)
processed_test = preprocessor_trx.transform(trx_test)

# Targets

In [8]:
train_target = spark.read.parquet('train_target.parquet')

In [9]:
preprocessor_target = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="mon",
    event_time_transformation="dt_to_timestamp",
    cols_identity=["target_1", "target_2", "target_3", "target_4"],
)
processed_target = preprocessor_target.fit_transform(train_target).drop('event_time')

In [10]:
processed_train = processed_train.join(processed_target, processed_target.client_id == processed_train.client_id, 'inner').drop(processed_target.client_id)

In [11]:
processed_train.write.parquet('trx_train_prepr.parquet', mode='overwrite')
processed_test.write.parquet('trx_test_prepr.parquet', mode='overwrite')